## Tensorflow


This chapter covers the usage of tensorflow, starting with basic usage, to implementing a the handwriting recognition with MNIST dataset.


In [ ]:
!pip install tensorflow
!pip install tensorflow_datasets

In [20]:
import tensorflow as tf
import tensorflow_datasets as tfds

print(tf.__version__)
assert tf.__version__ == "2.8.0"

2.8.0


In [21]:
(ds_train, ds_test), ds_info = tfds.load(
    "mnist",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [22]:
ds_info

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_path='/Users/alextanhongpin/tensorflow_datasets/mnist/3.0.1',
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
)

In [23]:
def normalize_img(image, label):
    """Normalizes images: uint8 -> float32"""
    return tf.cast(image, tf.float32) / 255.0, label

In [24]:
# Build a training pipeline.
ds_train = ds_train.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [25]:
# Build an evaluation pipeline.

ds_test = ds_test.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [26]:
# Create and train the model.
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(10),
    ]
)
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6
469/469 [==============================] - 5s 4ms/step - loss: 0.3579 - sparse_categorical_accuracy: 0.9003 - val_loss: 0.2010 - val_sparse_categorical_accuracy: 0.9411
Epoch 2/6
469/469 [==============================] - 1s 3ms/step - loss: 0.1723 - sparse_categorical_accuracy: 0.9505 - val_loss: 0.1399 - val_sparse_categorical_accuracy: 0.9589
Epoch 3/6
469/469 [==============================] - 1s 3ms/step - loss: 0.1225 - sparse_categorical_accuracy: 0.9652 - val_loss: 0.1125 - val_sparse_categorical_accuracy: 0.9659
Epoch 4/6
469/469 [==============================] - 1s 3ms/step - loss: 0.0942 - sparse_categorical_accuracy: 0.9724 - val_loss: 0.1024 - val_sparse_categorical_accuracy: 0.9687
Epoch 5/6
469/469 [==============================] - 1s 3ms/step - loss: 0.0759 - sparse_categorical_accuracy: 0.9778 - val_loss: 0.0887 - val_sparse_categorical_accuracy: 0.9728
Epoch 6/6
469/469 [==============================] - 1s 3ms/step - loss: 0.0620 - sparse_categorical_accu